In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [21]:
#Загрузка и нормализация данных
#Сначала подключить диск с выгрузками!!!!
import pandas as pd
#Читаем данные
oldData = pd.read_excel("/content/drive/MyDrive/Работа РСХБ/Выгрузки/выгрузка_21.10 постарше.xlsx")
newData = pd.read_excel("/content/drive/MyDrive/Работа РСХБ/Выгрузки/выгрузка_28.10.xlsx")
#Нормируем данные
import numpy as np
oldData = oldData.astype({'ИНН организации': np.int})
newData = newData.astype({'ИНН организации': np.int})
oldData["Дата регистрации организации на платформе \"Свое фермерство \""] \
= pd.to_datetime(oldData["Дата регистрации организации на платформе \"Свое фермерство \""])
newData["Дата регистрации организации на платформе \"Свое фермерство \""] \
= pd.to_datetime(newData["Дата регистрации организации на платформе \"Свое фермерство \""])
#Дата, с которой сравниваем
prevDate = pd.to_datetime("10.21.2021")#Месяц, число, год
#Дата формирования отчёта
nowDate = pd.to_datetime("10.28.2021")

In [26]:
#Прочитаем данные
excelOrder = pd.read_excel(
    "/content/drive/MyDrive/Работа РСХБ/Копия Наполнение каталога.xlsx",
    sheet_name = None                
)
#Удалим строку со статистикой, заполним пустые ячейки нулями, приведём ИНН к инту
excelOrder['ЖИВОТНЫЕ'] = (excelOrder['ЖИВОТНЫЕ'].drop(index=[0]).fillna(value=0)).astype({'ИНН организации': np.int}).reset_index(drop=True)
excelOrder['ПОСАД МАТЕР'] = (excelOrder['ПОСАД МАТЕР'].drop(index=[0]).fillna(value=0)).astype({'ИНН организации': np.int}).reset_index(drop=True)
excelOrder['СЕМЕНА'] = (excelOrder['СЕМЕНА'].drop(index=[0]).fillna(value=0)).astype({'ИНН организации': np.int}).reset_index(drop=True)

In [27]:
excelOrder['ЖИВОТНЫЕ']

,ИНН организации,Дата регистрации,Наименование региона регистрации,Наименование организации,2021-09-09 00:00:00,2021-09-16 00:00:00,2021-09-22 00:00:00,2021-07-10 00:00:00,2021-10-14 00:00:00,2021-10-21 00:00:00
0,10301865942,2020-04-10 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Бр...,1,1,1,1,1,1
1,10301975409,2020-10-16 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Па...,1,1,1,1,1,1
2,10300780848,2020-12-16 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Зе...,1,1,1,1,1,1
3,10104634015,2020-12-18 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Ки...,4,4,4,4,4,4
4,403002956,2020-09-09 00:00:00,Алтай,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МЕРКИТ""",1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
612,6372013018,2021-10-13 00:00:00,Самарская,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЗАРЯ""",0,0,0,0,5,5
613,6907011630,2021-08-31 00:00:00,Тверская,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПОЗИ...",0,0,0,0,3,3
614,171300542853,2020-10-21 00:00:00,Тыва,Глава крестьянского (фермерского) хозяйства Ор...,0,0,0,0,2,2
615,244309619203,13.10.2021,Красноярский,Глава крестьянского (фермерского) хозяйства Ма...,0,0,0,0,0,5


In [ ]:
# newSeedlings = newData[(newData['Категория товара уровня 1'] == "Посадочные материалы")].drop(
#             columns=[
#                      "Дата регистрации организации на платформе \"Свое фермерство \"",
#                      "Идентификатор организации",
#                      "Признак b2b",
#                      "Признак b2c",
#                      "Категория товара уровня 1"
#             ])
# refreshSeedlings = pd.merge(
#     excelOrder['ПОСАД МАТЕР'],
#     newSeedlings,
#     how="outer",
#     on=[
#         "ИНН организации", 
#         "Наименование региона регистрации",
#     ]
# ).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
# refreshSeedlings['Наименование организации'] = [x if x != "" else refreshSeedlings['Наименование организации_x']\
#                                                 for x in refreshSeedlings['Наименование организации_y']] 
# refreshSeedlings['Наименование организации_x'] = refreshSeedlings["Наименование организации"]
# refreshSeedlings = refreshSeedlings.drop(columns=["Наименование организации_y", "Наименование организации"])
# refreshSeedlings = refreshSeedlings.rename(columns={'Наименование организации_x':'Наименование организации'})
# refreshSeedlings

In [29]:
#Обновим листы для животных, семян, посадочных материалов

#Животные
newAnimals = newData[(newData['Категория товара уровня 1'] == "Животные") \
                       | (newData['Категория товара уровня 1'] == "Сельскохозяйственные животные")].drop(
            columns=[
                    #  "Дата регистрации организации на платформе \"Свое фермерство \"",
                     "Идентификатор организации",
                     "Признак b2b",
                     "Признак b2c",
                     "Категория товара уровня 1"
            ])
refreshAnimals = pd.merge(
    excelOrder['ЖИВОТНЫЕ'],
    newAnimals,
    how="outer",
    on=[
        "ИНН организации", 
        "Наименование региона регистрации",
    ]
).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
refreshAnimals['Наименование организации'] = [x if x != "" else refreshAnimals['Наименование организации_x']\
                                                for x in refreshAnimals['Наименование организации_y']] 
refreshAnimals['Наименование организации_x'] = refreshAnimals["Наименование организации"]
refreshAnimals = refreshAnimals.drop(columns=["Наименование организации_y", "Наименование организации"])
refreshAnimals = refreshAnimals.rename(columns={'Наименование организации_x':'Наименование организации'})
#Вставим строку со статистической информацией
titles = refreshAnimals.columns.values[3:len(refreshAnimals.columns.values)]
for i in range(len(titles)):
  if titles[i] == "Дата регистрации организации на платформе \"Свое фермерство \"":
    titles = np.delete(titles, i)
    break
tmpDict = {}
for title in titles:
  tmpDict[title] = [refreshAnimals[title].sum()]
tmpDict["Наименование организации"] = len(refreshAnimals["ИНН организации"].unique())
statisRow = pd.DataFrame(data=tmpDict, columns=refreshAnimals.columns.values)
statisRow.fillna(value="")
refreshAnimals = pd.concat([statisRow, refreshAnimals]).fillna(value="").reset_index(drop=True)
#Поставим колонку с датой регистрации в нормальное место
# refreshAnimals.insert(1, "Дата регистрации", refreshAnimals["Дата регистрации организации на платформе \"Свое фермерство \""])
# refreshAnimals = refreshAnimals.drop(columns=["Дата регистрации организации на платформе \"Свое фермерство \""])


#Семена
# newSeeds = newData[(newData['Категория товара уровня 1'] == "Семена")].drop(
#             columns=[
#                     #  "Дата регистрации организации на платформе \"Свое фермерство \"",
#                      "Идентификатор организации",
#                      "Признак b2b",
#                      "Признак b2c",
#                      "Категория товара уровня 1"
#             ])
# refreshSeeds = pd.merge(
#     excelOrder['СЕМЕНА'],
#     newSeeds,
#     how="outer",
#     on=[
#         "ИНН организации", 
#         "Наименование региона регистрации",
#     ]
# ).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
# refreshSeeds['Наименование организации'] = [x if x != "" else refreshSeeds['Наименование организации_x']\
#                                                 for x in refreshSeeds['Наименование организации_y']] 
# refreshSeeds['Наименование организации_x'] = refreshSeeds["Наименование организации"]
# refreshSeeds = refreshSeeds.drop(columns=["Наименование организации_y", "Наименование организации"])
# refreshSeeds = refreshSeeds.rename(columns={'Наименование организации_x':'Наименование организации'})
# #Вставим строку со статистической информацией
# titles = refreshSeeds.columns.values[3:len(refreshSeeds.columns.values)]
# for i in range(len(titles)):
#   if titles[i] == "Дата регистрации организации на платформе \"Свое фермерство \"":
#     titles = np.delete(titles, i)
#     break
# tmpDict = {}
# for title in titles:
#   tmpDict[title] = [refreshSeeds[title].sum()]
# tmpDict["Наименование организации"] = len(refreshSeeds["ИНН организации"].unique())
# statisRow = pd.DataFrame(data=tmpDict, columns=refreshSeeds.columns.values)
# statisRow.fillna(value="")
# refreshSeeds = pd.concat([statisRow, refreshSeeds]).fillna(value="").reset_index(drop=True)
# #Поставим колонку с датой регистрации в нормальное место
# # refreshSeeds.insert(1, "Дата регистрации", refreshSeeds["Дата регистрации организации на платформе \"Свое фермерство \""])
# # refreshSeeds = refreshSeeds.drop(columns=["Дата регистрации организации на платформе \"Свое фермерство \""])


# #Посадочные материалы
# newSeedlings = newData[(newData['Категория товара уровня 1'] == "Посадочные материалы")].drop(
#             columns=[
#                     #  "Дата регистрации организации на платформе \"Свое фермерство \"",
#                      "Идентификатор организации",
#                      "Признак b2b",
#                      "Признак b2c",
#                      "Категория товара уровня 1"
#             ])
# refreshSeedlings = pd.merge(
#     excelOrder['ПОСАД МАТЕР'],
#     newSeedlings,
#     how="outer",
#     on=[
#         "ИНН организации", 
#         "Наименование региона регистрации",
#     ]
# ).fillna(value=0).drop_duplicates(keep='first').rename(columns={'Количество товаров у поставщика': nowDate})
# refreshSeedlings['Наименование организации'] = [x if x != "" else refreshSeedlings['Наименование организации_x']\
#                                                 for x in refreshSeedlings['Наименование организации_y']] 
# refreshSeedlings['Наименование организации_x'] = refreshSeedlings["Наименование организации"]
# refreshSeedlings = refreshSeedlings.drop(columns=["Наименование организации_y", "Наименование организации"])
# refreshSeedlings = refreshSeedlings.rename(columns={'Наименование организации_x':'Наименование организации'})
# #Вставим строку со статистической информацией
# titles = refreshSeedlings.columns.values[3:len(refreshSeedlings.columns.values)]
# for i in range(len(titles)):
#   if titles[i] == "Дата регистрации организации на платформе \"Свое фермерство \"":
#     titles = np.delete(titles, i)
#     break
# tmpDict = {}
# for title in titles:
#   tmpDict[title] = [refreshSeedlings[title].sum()]
# tmpDict["Наименование организации"] = len(refreshSeedlings["ИНН организации"].unique())
# statisRow = pd.DataFrame(data=tmpDict, columns=refreshSeedlings.columns.values)
# statisRow.fillna(value="")
# refreshSeedlings = pd.concat([statisRow, refreshSeedlings]).fillna(value="").reset_index(drop=True)
#Поставим колонку с датой регистрации в нормальное место
# refreshSeedlings.insert(1, "Дата регистрации", refreshSeedlings["Дата регистрации организации на платформе \"Свое фермерство \""])
# refreshSeedlings = refreshSeedlings.drop(columns=["Дата регистрации организации на платформе \"Свое фермерство \""])


In [30]:
refreshAnimals

,ИНН организации,Дата регистрации,Наименование региона регистрации,Наименование организации,2021-09-09 00:00:00,2021-09-16 00:00:00,2021-09-22 00:00:00,2021-07-10 00:00:00,2021-10-14 00:00:00,2021-10-21 00:00:00,"Дата регистрации организации на платформе ""Свое фермерство """,2021-10-28 00:00:00
0,,,,623,1260.0,1300.0,1335.0,1371.0,1382.0,1391.0,,1414
1,10301865942,2020-04-10 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Бр...,1.0,1.0,1.0,1.0,1.0,1.0,2020-04-10 00:00:00,1
2,10301975409,2020-10-16 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Па...,1.0,1.0,1.0,1.0,1.0,1.0,2020-10-16 00:00:00,1
3,10300780848,2020-12-16 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Зе...,1.0,1.0,1.0,1.0,1.0,1.0,2020-12-16 00:00:00,1
4,10104634015,2020-12-18 00:00:00,Адыгея,Глава крестьянского (фермерского) хозяйства Ки...,4.0,4.0,4.0,4.0,4.0,4.0,2020-12-18 00:00:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...
619,613901928730,0,Ростовская,Глава крестьянского (фермерского) хозяйства Еф...,0.0,0.0,0.0,0.0,0.0,0.0,2021-10-20 00:00:00,4
620,240900016237,0,Красноярский,Глава крестьянского (фермерского) хозяйства Ге...,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-22 00:00:00,5
621,81100524868,0,Калмыкия,Глава крестьянского (фермерского) хозяйства Шу...,0.0,0.0,0.0,0.0,0.0,0.0,2021-10-27 00:00:00,5
622,3917508263,0,Калининградская,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ФЕРМА""",0.0,0.0,0.0,0.0,0.0,0.0,2021-10-20 00:00:00,5


In [ ]:
# # refreshSeeds['ИНН организации'].unique()
# refreshSeeds
with pd.ExcelWriter(f"/content/drive/MyDrive/Работа РСХБ/Общая папка/Наполнение каталога {nowDate.strftime('%d.%m-%Y')}.xlsx") as writer:  

    refreshSeeds.to_excel(writer, sheet_name='СЕМЕНА',index=False)

    refreshSeedlings.to_excel(writer, sheet_name='ПОСАД МАТЕР',index=False)
    
    refreshAnimals.to_excel(writer, sheet_name='ЖИВОТНЫЕ',index=False)

In [23]:
print("КАТЕГОРИЯ СЕЛЬХОЗ-ЖИВОТНЫЕ")
oldDataINN = oldData[(oldData['Категория товара уровня 1'] == "Животные") \
                       | (oldData['Категория товара уровня 1'] == "Сельскохозяйственные животные")]["ИНН организации"].unique()
newDataINN = newData[(newData['Категория товара уровня 1'] == "Животные") \
                       | (newData['Категория товара уровня 1'] == "Сельскохозяйственные животные")]["ИНН организации"].unique() 
dif = np.setdiff1d(newDataINN, oldDataINN)#появились
print(f"Было {len(oldDataINN)} производителей и поставщиков")
print(f"Стало {len(newDataINN)} производителей и поставщиков")
print(f"Прирост составил {len(newDataINN) - len(oldDataINN)} производителей и поставщиков")
newSellers = newData[(newData["ИНН организации"].isin(dif)) & ((newData['Категория товара уровня 1'] == "Животные") \
                       | (newData['Категория товара уровня 1'] == "Сельскохозяйственные животные"))]
print(f"За данный период в категории с/х животных появилось {newSellers.shape[0]} новых продавца или производителя.")

newSellersCount = len(newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]])
activatedSellers = newSellers.shape[0] - newSellersCount
print(f"Из них новых продавцов {newSellersCount}, активизировавшихся продавцов {activatedSellers}")

dif = np.setdiff1d(oldDataINN, newDataINN)#пропали
print(f"Также за период из категории с/х животных ушло {len(dif)} продавцов и производителей")
oldSum = oldData[(oldData['Категория товара уровня 1'] == "Животные") \
                       | (oldData['Категория товара уровня 1'] == "Сельскохозяйственные животные")]["Количество товаров у поставщика"].sum()
newSum = newData[(newData['Категория товара уровня 1'] == "Животные") \
                       | (newData['Категория товара уровня 1'] == "Сельскохозяйственные животные")]["Количество товаров у поставщика"].sum()
print(f"В категории было {oldSum} товаров, стало {newSum} товаров. Прирост составил {newSum - oldSum}.")
#Сколько добавили новые участники
newcomersContrib = newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["Количество товаров у поставщика"].sum()
print(f"Из {newSum - oldSum} новых товаров {newcomersContrib} добавили новые участники площадки.")
#Список новых участников
print()
print("НОВЫЕ УЧАСТНИКИ")
for inn in (newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["ИНН организации"]):
  name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
  print(f"{inn} {name[0]}")
#Список активизировавшихся участников
print()
print("АКТИВИЗИРОВАВШИЕСЯ УЧАСТНИКИ")
for inn in (newSellers[prevDate >= newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["ИНН организации"]):
  name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
  print(f"{inn} {name[0]}")

КАТЕГОРИЯ СЕЛЬХОЗ-ЖИВОТНЫЕ
Было 617 производителей и поставщиков
Стало 623 производителей и поставщиков
Прирост составил 6 производителей и поставщиков
За данный период в категории с/х животных появилось 6 новых продавца или производителя.
Из них новых продавцов 1, активизировавшихся продавцов 5
Также за период из категории с/х животных ушло 0 продавцов и производителей
В категории было 1391 товаров, стало 1414 товаров. Прирост составил 23.
Из 23 новых товаров 5 добавили новые участники площадки.

НОВЫЕ УЧАСТНИКИ
81100524868 Глава крестьянского (фермерского) хозяйства Шукургалиев Кобланды Захарович

АКТИВИЗИРОВАВШИЕСЯ УЧАСТНИКИ
613900202897 Глава крестьянского (фермерского) хозяйства Зубенко Андрей Николаевич
613901928730 Глава крестьянского (фермерского) хозяйства Ефремов Роман Алексеевич
240900016237 Глава крестьянского (фермерского) хозяйства Генза Олег Саламонович
3917508263 ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФЕРМА"
3849012202 ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ХАДАЙС

In [ ]:
newSellers

In [24]:
print("КАТЕГОРИЯ СЕМЕНА")
oldDataINN = oldData[(oldData['Категория товара уровня 1'] == "Семена")]["ИНН организации"].unique()
newDataINN = newData[(newData['Категория товара уровня 1'] == "Семена")]["ИНН организации"].unique() 
dif = np.setdiff1d(newDataINN, oldDataINN)#появились
print(f"Было {len(oldDataINN)} производителей и поставщиков")
print(f"Стало {len(newDataINN)} производителей и поставщиков")
print(f"Прирост составил {len(newDataINN) - len(oldDataINN)} производителей и поставщиков")
newSellers = newData[(newData["ИНН организации"].isin(dif)) & ((newData['Категория товара уровня 1'] == "Семена"))]
print(f"За данный период в категории семян появилось {newSellers.shape[0]} новых продавца или производителя.")

newSellersCount = len(newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]])
activatedSellers = newSellers.shape[0] - newSellersCount
print(f"Из них новых продавцов {newSellersCount}, активизировавшихся продавцов {activatedSellers}")

dif = np.setdiff1d(oldDataINN, newDataINN)#пропали
print(f"Также за период из категории семян ушло {len(dif)} продавцов и производителей")
oldSum = oldData[(oldData['Категория товара уровня 1'] == "Семена")]["Количество товаров у поставщика"].sum()
newSum = newData[(newData['Категория товара уровня 1'] == "Семена")]["Количество товаров у поставщика"].sum()
print(f"В категории было {oldSum} товаров, стало {newSum} товаров. Прирост составил {newSum - oldSum}")
#Сколько добавили новые участники
newcomersContrib = newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["Количество товаров у поставщика"].sum()
print(f"Из {newSum - oldSum} новых товаров {newcomersContrib} добавили новые участники площадки.")
#Список новых участников
print()
print("НОВЫЕ УЧАСТНИКИ")
for inn in (newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["ИНН организации"]):
  name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
  print(f"{inn} {name[0]}")
#Список активизировавшихся участников
print()
print("АКТИВИЗИРОВАВШИЕСЯ УЧАСТНИКИ")
for inn in (newSellers[prevDate >= newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["ИНН организации"]):
  name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
  print(f"{inn} {name[0]}")

КАТЕГОРИЯ СЕМЕНА
Было 1606 производителей и поставщиков
Стало 1624 производителей и поставщиков
Прирост составил 18 производителей и поставщиков
За данный период в категории семян появилось 18 новых продавца или производителя.
Из них новых продавцов 11, активизировавшихся продавцов 7
Также за период из категории семян ушло 0 продавцов и производителей
В категории было 7602 товаров, стало 7642 товаров. Прирост составил 40
Из 40 новых товаров 16 добавили новые участники площадки.

НОВЫЕ УЧАСТНИКИ
790601046479 Глава крестьянского (фермерского) хозяйства Карипова Елена Ефимовна
563805805809 Глава крестьянского (фермерского) хозяйства Кадиев Артур Абдуразакович
611700180900 Глава крестьянского (фермерского) хозяйства Парфененко Николай Васильевич
613600531107 Глава крестьянского (фермерского) хозяйства Чернявская Галина Ивановна
613901060170 Глава крестьянского (фермерского) хозяйства Магомедов Габибулла Магомедгаджиевич
612704339501 Глава крестьянского (фермерского) хозяйства Харишман Викт

In [25]:
print("КАТЕГОРИЯ ПОСАДОЧНЫЕ МАТЕРИАЛЫ")
oldDataINN = oldData[(oldData['Категория товара уровня 1'] == "Посадочные материалы")]["ИНН организации"].unique()
newDataINN = newData[(newData['Категория товара уровня 1'] == "Посадочные материалы")]["ИНН организации"].unique() 
dif = np.setdiff1d(newDataINN, oldDataINN)#появились
print(f"Было {len(oldDataINN)} производителей и поставщиков")
print(f"Стало {len(newDataINN)} производителей и поставщиков")
print(f"Прирост составил {len(newDataINN) - len(oldDataINN)} производителей и поставщиков")
newSellers = newData[(newData["ИНН организации"].isin(dif)) & ((newData['Категория товара уровня 1'] == "Посадочные материалы"))]
print(f"За данный период в категории посадочных материалов появилось {newSellers.shape[0]} новых продавца или производителя.")

newSellersCount = len(newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]])
activatedSellers = newSellers.shape[0] - newSellersCount
print(f"Из них новых продавцов {newSellersCount}, активизировавшихся продавцов {activatedSellers}")

dif = np.setdiff1d(oldDataINN, newDataINN)#пропали
print(f"Также за период из категории посадочных материалов ушло {len(dif)} продавцов и производителей")
oldSum = oldData[(oldData['Категория товара уровня 1'] == "Посадочные материалы")]["Количество товаров у поставщика"].sum()
newSum = newData[(newData['Категория товара уровня 1'] == "Посадочные материалы")]["Количество товаров у поставщика"].sum()
print(f"В категории было {oldSum} товаров, стало {newSum} товаров. Прирост составил {newSum - oldSum}")
#Сколько добавили новые участники
newcomersContrib = newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["Количество товаров у поставщика"].sum()
print(f"Из {newSum - oldSum} новых товаров {newcomersContrib} добавили новые участники площадки.")
#Список новых участников
print()
print("НОВЫЕ УЧАСТНИКИ")
for inn in (newSellers[prevDate < newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["ИНН организации"]):
  name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
  print(f"{inn} {name[0]}")
#Список активизировавшихся участников
print()
print("АКТИВИЗИРОВАВШИЕСЯ УЧАСТНИКИ")
for inn in (newSellers[prevDate >= newSellers["Дата регистрации организации на платформе \"Свое фермерство \""]]["ИНН организации"]):
  name = newSellers.loc[newSellers["ИНН организации"] == inn]["Наименование организации"].values
  print(f"{inn} {name[0]}")

КАТЕГОРИЯ ПОСАДОЧНЫЕ МАТЕРИАЛЫ
Было 78 производителей и поставщиков
Стало 78 производителей и поставщиков
Прирост составил 0 производителей и поставщиков
За данный период в категории посадочных материалов появилось 0 новых продавца или производителя.
Из них новых продавцов 0, активизировавшихся продавцов 0
Также за период из категории посадочных материалов ушло 0 продавцов и производителей
В категории было 1021 товаров, стало 1021 товаров. Прирост составил 0
Из 0 новых товаров 0 добавили новые участники площадки.

НОВЫЕ УЧАСТНИКИ

АКТИВИЗИРОВАВШИЕСЯ УЧАСТНИКИ
